## Configure local LLM coz OpenAI rate limits suck!

In [1]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.prompts import PromptTemplate
from huggingface_hub import login
from typing import Any, Dict
from dotenv import load_dotenv
import torch
import os
load_dotenv()

token = os.environ["HF_API_KEY"]
login(token)
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    torch_dtype="auto",
    device_map="auto",
)

# Set up Hugging Face pipeline for batched inference
hf_llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    batch_size=4,  # Adjust based on GPU memory
)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-27 02:38:33.653411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737945513.671048    3171 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737945513.676441    3171 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]
Device set to use cuda:0


In [3]:
from transformers import pipeline
from datasets import Dataset
# from promptlib import PromptTemplate  # Assuming you're using a custom prompt library

def get_completions(paragraphs, hf_llm):
    # Generate prompts for all paragraphs
    prompts = []
    for paragraph in paragraphs:
        template = f"""
        <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an expert event extraction system. Analyze the given paragraph and list all significant events in JSON format. For each event, include:
        1. "event_type" (e.g., political, natural disaster, scientific discovery)
        2. "event_summary" (concise 5-10 word description)
        3. "explanation" (context/reason for significance)
        Output must be valid JSON. No explanations outside JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>
        Paragraph: {paragraph}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
        prompts.append(template)

    # Batch inference
    outputs = hf_llm(
        prompts,
        max_new_tokens=500,  # Adjust based on response length
        pad_token_id=tokenizer.eos_token_id,  # Required for padding
        temperature=0.3,
    )

    # Extract generated text
    return [output[0]["generated_text"] for output in outputs]

In [4]:
paragraphs = [
    "The president announced new climate policies yesterday...",
    "A major earthquake struck Japan earlier today...",
]

hf_llm.tokenizer.pad_token_id = model.config.eos_token_id[0]
results = get_completions(paragraphs, hf_llm)
for result in results:
    print(result)  # JSON outputs for all paragraphs


        <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an expert event extraction system. Analyze the given paragraph and list all significant events in JSON format. For each event, include:
        1. "event_type" (e.g., political, natural disaster, scientific discovery)
        2. "event_summary" (concise 5-10 word description)
        3. "explanation" (context/reason for significance)
        Output must be valid JSON. No explanations outside JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>
        Paragraph: The president announced new climate policies yesterday...<|eot_id|><|start_header_id|>assistant<|end_header_id|>

```
[
  {
    "event_type": "political",
    "event_summary": "President announces new climate policies",
    "explanation": "Government action to address climate change"
  }
]
```

        <|begin_of_text|><|start_header_id|>system<|end_header_id|>
        You are an expert event extraction system. Analyze the given paragraph an

In [14]:
# Initialize pipeline once (not per-call)
hf_llm = pipeline(
    "text-generation",
    model="your-model-name",
    device=0,  # GPU device
    torch_dtype="auto"
)

# Process multiple paragraphs
paragraphs = [para1, para2, para3, ...]  # Your input texts
results = generate_batch_completion(paragraphs, hf_llm, batch_size=2)

Device set to use cuda:0


In [4]:
import PyPDF2
import llama_parse
from datetime import datetime
from llama_parse import LlamaParse
from all_imports import *
import nest_asyncio
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage

nest_asyncio.apply()

parser = LlamaParse(
    api_key=os.environ["LLAMA_INDEX_API_KEY"],  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="text",  # "markdown" and "text" are available,
    chunk_size="paragraph",
    num_workers=8,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
)

# sync
# documents = parser.load_data("beige_book_pdfs/2024/BeigeBook_20240117.pdf")

In [ ]:
import os
from pathlib import Path

def read_year_files(year, base_dir="."):
    """
    Read all PDF files from a specific year directory and return their contents.
    
    Args:
        year (int/str): Year to read files from
        base_dir (str): Base directory containing year folders (default: current directory)
    
    Returns:
        dict: Dictionary with filenames as keys and text content as values
    """
    year_dir = Path(base_dir) / str(year)
    
    if not year_dir.exists():
        raise FileNotFoundError(f"No directory found for year {year}")
        
    if not year_dir.is_dir():
        raise NotADirectoryError(f"{year_dir} is not a directory")
    
    pdf_files = list(year_dir.glob("*.pdf"))
    contents = {}
    
    for pdf_file in pdf_files:
        try:
            print(pdf_file)
            contents[pdf_file.name] = parser.load_data(pdf_file)
            # documents = parser.load_data(pdf_file)
        except Exception as e:
            print(f"Error reading {pdf_file.name}: {str(e)}")
            contents[pdf_file.name] = None
    
    return contents

all_year_contents = []
# Read all files from 2020 directory
for year in range(2000, 2025):
    year_contents = read_year_files(year, base_dir="beige_book_pdfs")
    all_year_contents += year_contents



beige_book_pdfs/2000/fomc20000202beige20000119.pdf
Started parsing the file under job_id 0c6ae1e3-9a18-456b-ae58-66b7d86ff779
....

In [23]:

def get_completion(paragraph):
    ### Code to test the local LLM inference
    template = """
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert event extraction system. Analyze the given paragraph and list all significant events in JSON format. For each event, include:
    1. "event_type" (e.g., political, natural disaster, scientific discovery)
    2. "event_summary" (concise 5-10 word description)
    3. "explanation" (context/reason for significance)
    Output must be valid JSON. No explanations outside JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>
    Paragraph: {paragraph}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    prompt = PromptTemplate.from_template(template)
    
    chain = prompt | hf_llm
    
    return chain.invoke({"paragraph": paragraph})

In [21]:
data.id_

'fe624bff-884c-4573-a178-f122210efbfd'

In [27]:
events_list = []

for document in documents:
    response = get_completion(document.text)
    gen_output = response.split("<|start_header_id|>assistant<|end_header_id|>")[1]
    start_index = gen_output.find('[')
    end_index = gen_output.rfind(']') + 1  # +1 to include the `]` character
    
    try:
        # Slice the string to keep only the JSON array
        cleaned_str = gen_output[start_index:end_index]
        events = eval(cleaned_str)
        events_list += events
        print(f"total events  {document.id_}: {len(events)}")
    except: 
        end_index = gen_output.rfind('}')
        cleaned_str = gen_output[start_index:end_index] + "]"
        try:
            events = eval(cleaned_str)
            events_list += events
            print(f"total events for {document.id_}: {len(events)}")
        except:
            print("couldn't clean string, replacing with null", data.id_)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


total events  0a5a488d-201a-4fe2-bb5e-68882861adb3: 2


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


total events  8e17022c-14ed-4f21-ad2a-4d1b500e57e9: 12


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


total events  6e82adca-dbae-4262-b4bd-af4caf726431: 4


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


total events  540551ee-f470-478b-ac77-c2ce05941a88: 8


KeyboardInterrupt: 

In [12]:
response

'\n<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nYou are an expert event extraction system. Analyze the given paragraph and list all significant events in JSON format. For each event, include:\n1. "event_type" (e.g., political, natural disaster, scientific discovery)\n2. "event_summary" (concise 5-10 word description)\n3. "explanation" (context/reason for significance)\nOutput must be valid JSON. No explanations outside JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>\nParagraph: 8\n\n                           Federal Reserve Bank of\n                           New York\nSummary of Economic Activity\nEconomic activity in the Second District declined slightly during the latest reporting period, con-\ntinuing a period of sustained weakness. Labor market conditions continued to cool but remained\nsolid. Employment growth slowed to a slight pace, as demand for labor softened with economic\nuncertainty inhibiting hiring plans. Inflationary pressures were little change